In [1]:
%autosave 20

import pandas as pd
import numpy as np
import collections
from collections import defaultdict

# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Autosaving every 20 seconds


In [2]:
df = pd.read_excel('Database_2019_2021.xlsx', index_col=0,
                  dtype={'fipsValue':str, 'Year':str, 'Month':str})

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56160 entries, 0 to 56159
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               56160 non-null  object 
 1   County              56160 non-null  object 
 2   fipsValue           56160 non-null  object 
 3   Population Density  56160 non-null  float64
 4   Population          56160 non-null  int64  
 5   Region              56160 non-null  object 
 6   Year                56160 non-null  object 
 7   Month               56160 non-null  object 
 8   Disaster            56160 non-null  int64  
 9   Number of Workers   56160 non-null  int64  
 10  SNAP_Applications   56160 non-null  int64  
 11  Searchterm          56160 non-null  object 
 12  Search              56160 non-null  float64
dtypes: float64(2), int64(4), object(7)
memory usage: 6.0+ MB


In [4]:
df1 = df.copy()

### Make each term as a column



In [6]:
# create empty cols with search term

search = df1['Searchterm'].unique()

for word in search:
    df1[word] = 0
    
df1.head()

,State,County,fipsValue,Population Density,Population,Region,Year,Month,Disaster,Number of Workers,SNAP_Applications,Searchterm,Search,calfresh,food bank,food pantry,food stamps,supplemental nutrition assistance program,snap
0,California,Alameda County,06001,1898.5,1559308,San Francisco-Oakland-San Jose CA,2019,1,0,45,5515,calfresh,24.00,0,0,0,0,0,0
1,California,Alameda County,06001,1898.5,1559308,San Francisco-Oakland-San Jose CA,2019,1,0,45,5515,food bank,28.75,0,0,0,0,0,0
2,California,Alameda County,06001,1898.5,1559308,San Francisco-Oakland-San Jose CA,2019,1,0,45,5515,food pantry,38.00,0,0,0,0,0,0
3,California,Alameda County,06001,1898.5,1559308,San Francisco-Oakland-San Jose CA,2019,1,0,45,5515,food stamps,55.50,0,0,0,0,0,0
4,California,Alameda County,06001,1898.5,1559308,San Francisco-Oakland-San Jose CA,2019,1,0,45,5515,supplemental nutrition assistance program,54.50,0,0,0,0,0,0


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56160 entries, 0 to 56159
Data columns (total 19 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   State                                      56160 non-null  object 
 1   County                                     56160 non-null  object 
 2   fipsValue                                  56160 non-null  object 
 3   Population Density                         56160 non-null  float64
 4   Population                                 56160 non-null  int64  
 5   Region                                     56160 non-null  object 
 6   Year                                       56160 non-null  object 
 7   Month                                      56160 non-null  object 
 8   Disaster                                   56160 non-null  int64  
 9   Number of Workers                          56160 non-null  int64  
 10  SNAP_Applications     

In [8]:
# word : idx in database

lookup = {
    'calfresh':-6,
    'food pantry':-4,
    'food bank':-5,
    'supplemental nutrition assistance program':-2,
    'food stamps':-3,
    'snap' : -1
}

In [9]:
for i in range(len(df1)):
    term = df1.iloc[i, 11] # the idx of Searchterm
    value = df1.iloc[i,12] # the idx of search
    idx = lookup[term]
    df1.iloc[i, idx] = value
    
df1.head()

,State,County,fipsValue,Population Density,Population,Region,Year,Month,Disaster,Number of Workers,SNAP_Applications,Searchterm,Search,calfresh,food bank,food pantry,food stamps,supplemental nutrition assistance program,snap
0,California,Alameda County,06001,1898.5,1559308,San Francisco-Oakland-San Jose CA,2019,1,0,45,5515,calfresh,24.00,24.0,0.00,0.0,0.0,0.0,0.0
1,California,Alameda County,06001,1898.5,1559308,San Francisco-Oakland-San Jose CA,2019,1,0,45,5515,food bank,28.75,0.0,28.75,0.0,0.0,0.0,0.0
2,California,Alameda County,06001,1898.5,1559308,San Francisco-Oakland-San Jose CA,2019,1,0,45,5515,food pantry,38.00,0.0,0.00,38.0,0.0,0.0,0.0
3,California,Alameda County,06001,1898.5,1559308,San Francisco-Oakland-San Jose CA,2019,1,0,45,5515,food stamps,55.50,0.0,0.00,0.0,55.5,0.0,0.0
4,California,Alameda County,06001,1898.5,1559308,San Francisco-Oakland-San Jose CA,2019,1,0,45,5515,supplemental nutrition assistance program,54.50,0.0,0.00,0.0,0.0,54.5,0.0


In [10]:
state = df1.State.unique()
county = df1.County.unique()
year = df1.Year.unique()
month = df1.Month.unique()

# create empty dateframe to save the new data

col = df1.columns.tolist()
new = pd.DataFrame(columns=col)

In [11]:
count = 1

while count > 0:
    for s in state:
        for c in county:
            for y in year:
                for m in month:
                    cur_table = df1.loc[(df.State == s) & (df1.County == c) & (df1.Year == y) & (df1.Month == m)].reset_index()

                    if len(cur_table) != 0 :

                        # first part
                        first = cur_table.iloc[0, 0:13].reset_index().T # first 13 need to keep
                        first.columns = first.iloc[0]
                        first = first.drop(index='index')
                        first = first.drop(columns='index')


                        # second part use T
                        second = cur_table.groupby(['State','County','fipsValue', 'Population Density','Population','Region', 'Year','Month','Disaster','Number of Workers','SNAP_Applications','Searchterm'])[('Search')].max().reset_index()
                        second = second.iloc[:, -2::].T # 


                        # make first row to col
                        second.columns = second.iloc[0]
                        second = second.drop(index='Searchterm').reset_index(drop=True)




                        # combine two by creating cols
                        for col in second.columns.to_list():
                            first[col] = second[col]

                        new = new.merge(first, how='outer')
                        
    count -= 1



/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when

...

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when

In [12]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11232 entries, 0 to 11231
Data columns (total 19 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   Search                                     0 non-null      object
 1   snap                                       9144 non-null   object
 2   State                                      11232 non-null  object
 3   County                                     11232 non-null  object
 4   fipsValue                                  11232 non-null  object
 5   Population Density                         11232 non-null  object
 6   Population                                 11232 non-null  object
 7   Region                                     11232 non-null  object
 8   Year                                       11232 non-null  object
 9   Month                                      11232 non-null  object
 10  Disaster                          

In [13]:
# drop two col

new = new.drop(columns=['Searchterm', 'Search'])

In [14]:
new.fillna(0, inplace=True)

In [15]:
# replace the col snap to the end

# Get the DataFrame column names as a list
clist = list(new.columns)

# Rearrange list the way you like 
clist_new = clist[1::] + clist[:1]  # brings the first column in the last place

new = new[clist_new]

In [16]:
# rename the col for convenience

new.rename(columns={"Population Density": "PDensity",\
                    'Number of Workers':'numberOfWorkers',\
                    'Disaster':'numberOfDisaster',\
                    'food bank':'google_food_bank',\
                    'food pantry':'google_food_pantry',\
                    'food stamps':'google_food_stamps',\
                    'supplemental nutrition assistance program':'google_supplemental',\
                    'snap':'google_snap'}, inplace = True)

In [17]:
new.head()

,State,County,fipsValue,PDensity,Population,Region,Year,Month,numberOfDisaster,numberOfWorkers,SNAP_Applications,calfresh,google_food_bank,google_food_pantry,google_food_stamps,google_supplemental,google_snap
0,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,1,0.0,45.0,5515.0,24.00,28.75,38.00,55.50,54.50,0.0
1,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,2,0.0,397.0,4478.0,27.25,27.75,38.75,31.25,36.75,0.0
2,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,3,0.0,191.0,5041.0,19.60,23.20,33.80,31.80,35.20,0.0
3,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,4,0.0,808.0,5253.0,23.25,22.50,35.50,22.00,29.25,0.0
4,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,5,0.0,502.0,8074.0,35.00,26.75,31.50,29.50,31.25,0.0


In [18]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11232 entries, 0 to 11231
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   State                11232 non-null  object 
 1   County               11232 non-null  object 
 2   fipsValue            11232 non-null  object 
 3   PDensity             11232 non-null  float64
 4   Population           11232 non-null  float64
 5   Region               11232 non-null  object 
 6   Year                 11232 non-null  object 
 7   Month                11232 non-null  object 
 8   numberOfDisaster     11232 non-null  float64
 9   numberOfWorkers      11232 non-null  float64
 10  SNAP_Applications    11232 non-null  float64
 11  calfresh             11232 non-null  float64
 12  google_food_bank     11232 non-null  float64
 13  google_food_pantry   11232 non-null  float64
 14  google_food_stamps   11232 non-null  float64
 15  google_supplemental  11232 non-null 

In [19]:
new.describe()

,PDensity,Population,numberOfDisaster,numberOfWorkers,SNAP_Applications,calfresh,google_food_bank,google_food_pantry,google_food_stamps,google_supplemental,google_snap
count,11232.000000,1.123200e+04,11232.000000,11232.000000,11232.000000,11232.000000,11232.000000,11232.000000,11232.000000,11232.000000,11232.000000
mean,171.535577,2.154244e+05,0.125267,79.467682,1278.686788,9.009184,27.240207,35.668381,33.186605,38.505084,54.093928
std,464.591814,7.495632e+05,0.464456,1036.900143,4898.776658,19.908096,13.373129,15.372075,12.351754,12.120201,27.935199
min,0.100000,6.400000e+01,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,12.600000,13.250000,0.000000
25%,7.075000,7.766750e+03,0.000000,0.000000,47.000000,0.000000,18.500000,24.750000,24.000000,28.250000,47.750000
50%,25.400000,2.184650e+04,0.000000,0.000000,158.500000,0.000000,24.750000,35.500000,31.600000,37.500000,65.000000
75%,99.650000,9.560425e+04,0.000000,0.000000,618.250000,0.000000,32.000000,46.600000,39.250000,46.750000,73.000000
max,3575.300000,9.974203e+06,3.000000,61507.000000,125900.000000,92.750000,94.250000,89.750000,80.500000,81.250000,92.000000


In [20]:
new2 = new.copy()

### Create datetime column as buffer

In [21]:
# transfer float to int

col_need = ['Year', 'Month']

for col in col_need:
    new2[col] = new2[col].astype(int)

In [22]:
new2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11232 entries, 0 to 11231
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   State                11232 non-null  object 
 1   County               11232 non-null  object 
 2   fipsValue            11232 non-null  object 
 3   PDensity             11232 non-null  float64
 4   Population           11232 non-null  float64
 5   Region               11232 non-null  object 
 6   Year                 11232 non-null  int64  
 7   Month                11232 non-null  int64  
 8   numberOfDisaster     11232 non-null  float64
 9   numberOfWorkers      11232 non-null  float64
 10  SNAP_Applications    11232 non-null  float64
 11  calfresh             11232 non-null  float64
 12  google_food_bank     11232 non-null  float64
 13  google_food_pantry   11232 non-null  float64
 14  google_food_stamps   11232 non-null  float64
 15  google_supplemental  11232 non-null 

In [23]:
# create date_time
from datetime import datetime
new2['date_time'] = 0

for i in range(len(new2)):
    cur_year = str(new2.iloc[i, 6]) # 6 is the index of year
    cur_mon = str(new2.iloc[i, 7]) # 7 is the index of mon
    cur_date_string = cur_year + ' ' + cur_mon
    cur_datetime_string = pd.to_datetime(cur_date_string)
    new2.iloc[i, -1] = cur_datetime_string # -1 is the index of date_time in this step
    
# create last_date_time
new2['last_date_time'] = 0

for i in range(len(new2)):
    dtDate = new2.iloc[i,-2] # -2 is the index of date_time in this step
    new2.iloc[i,-1] = dtDate - pd.DateOffset(months=1)  # -1 is the index of last_date_time in this step

In [24]:
new2.head()

,State,County,fipsValue,PDensity,Population,Region,Year,Month,numberOfDisaster,numberOfWorkers,SNAP_Applications,calfresh,google_food_bank,google_food_pantry,google_food_stamps,google_supplemental,google_snap,date_time,last_date_time
0,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,1,0.0,45.0,5515.0,24.00,28.75,38.00,55.50,54.50,0.0,2019-01-01 00:00:00,2018-12-01 00:00:00
1,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,2,0.0,397.0,4478.0,27.25,27.75,38.75,31.25,36.75,0.0,2019-02-01 00:00:00,2019-01-01 00:00:00
2,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,3,0.0,191.0,5041.0,19.60,23.20,33.80,31.80,35.20,0.0,2019-03-01 00:00:00,2019-02-01 00:00:00
3,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,4,0.0,808.0,5253.0,23.25,22.50,35.50,22.00,29.25,0.0,2019-04-01 00:00:00,2019-03-01 00:00:00
4,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,5,0.0,502.0,8074.0,35.00,26.75,31.50,29.50,31.25,0.0,2019-05-01 00:00:00,2019-04-01 00:00:00


In [25]:
new2.rename(columns={'calfresh':'google_calfresh'}, inplace=True)

In [27]:
new2.head()

,State,County,fipsValue,PDensity,Population,Region,Year,Month,numberOfDisaster,numberOfWorkers,SNAP_Applications,google_calfresh,google_food_bank,google_food_pantry,google_food_stamps,google_supplemental,google_snap,date_time,last_date_time
0,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,1,0.0,45.0,5515.0,24.00,28.75,38.00,55.50,54.50,0.0,2019-01-01 00:00:00,2018-12-01 00:00:00
1,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,2,0.0,397.0,4478.0,27.25,27.75,38.75,31.25,36.75,0.0,2019-02-01 00:00:00,2019-01-01 00:00:00
2,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,3,0.0,191.0,5041.0,19.60,23.20,33.80,31.80,35.20,0.0,2019-03-01 00:00:00,2019-02-01 00:00:00
3,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,4,0.0,808.0,5253.0,23.25,22.50,35.50,22.00,29.25,0.0,2019-04-01 00:00:00,2019-03-01 00:00:00
4,California,Alameda County,06001,1898.5,1559308.0,San Francisco-Oakland-San Jose CA,2019,5,0.0,502.0,8074.0,35.00,26.75,31.50,29.50,31.25,0.0,2019-05-01 00:00:00,2019-04-01 00:00:00


In [30]:
to_origin_jan = new2.loc[(new2.Month == 12) & (new2.Year == 2021)]

In [31]:
to_origin_jan.to_excel('to_origin_jan_June8.xlsx')

### Find last_snap, last_workder, last_disater, last_google words by the key [last_date_time, County and State]

In [ ]:
new2['last_snap'] = 0
new2['last_worker'] = 0
new2['last_disaster'] = 0
new2['last_google_calfresh'] = 0
new2['last_google_food_bank'] = 0
new2['last_google_food_pantry'] = 0
new2['last_google_food_stamps'] = 0
new2['last_google_supplemental'] = 0
new2['last_google_snap'] = 0

In [ ]:
# get the idx of each col
clist = list(new2.columns)
lookup = collections.defaultdict(int) # {colname:idx}
for i in range(len(clist)):
    lookup[clist[i]] = i

In [ ]:
new2.info()

In [ ]:
for i in range(len(new2)):
    state = new2.iloc[i, 0]
    county = new2.iloc[i, 1]
    last_date_time = new2.last_date_time[i]

    filtet_table = new2.loc[(new2.date_time == last_date_time) & (new2.State == state) & (new2.County == county)]

    
    if len(filtet_table) != 0:
        # use iloc[r, c] to set cells
        new2.iloc[i, lookup['last_disaster']] = filtet_table.numberOfDisaster.values[0]
        new2.iloc[i, lookup['last_worker']] = filtet_table.numberOfWorkers.values[0]
        new2.iloc[i, lookup['last_snap']] = filtet_table.SNAP_Applications.values[0]
        new2.iloc[i, lookup['last_google_calfresh']] = filtet_table.google_calfresh.values[0]
        new2.iloc[i, lookup['last_google_food_bank']] = filtet_table.google_food_bank.values[0]
        new2.iloc[i, lookup['last_google_food_pantry']] = filtet_table.google_food_pantry.values[0]
        new2.iloc[i, lookup['last_google_food_stamps']] = filtet_table.google_food_stamps.values[0]
        new2.iloc[i, lookup['last_google_supplemental']] = filtet_table.google_supplemental.values[0]
        new2.iloc[i, lookup['last_google_snap']] = filtet_table.google_snap.values[0]

In [ ]:
new2.info()

In [ ]:
new2.info()

## Data for 2022 prediction

In [ ]:
data_2021 = new2.loc[(new2.Year == 2021)]

In [ ]:
col_need_2021 = ['State', 'PDensity', 'Population', 'SNAP_Applications', 'numberOfWorkers', 'numberOfDisaster', 'google_calfresh', 'google_food_bank', 'google_food_pantry', 'google_food_stamps', 'google_supplemental', 'google_snap', 'Year', 'Month','date_time']

In [ ]:
data_2021 = data_2021[col_need_2021]

In [ ]:
# create dummy variables
dummies = pd.get_dummies(data_2021['State'], prefix='State')

# concatenate the dummy variables with the original dataframe
data_2021 = pd.concat([data_2021, dummies], axis=1)

In [ ]:
data_2021.info()

In [ ]:
# 3. Add seasonal dummy variables

# 3.1 add 'summer' -> whether last month is 6/7/8
# 3.2 add 'holiday' -> whether last month is 11/12
summer_month = [6, 7, 8]
holiday_month = [11, 12]


data_2021['summer'] = 0    # idx= -2
data_2021['holiday'] = 0   # idx= -1

for i in range(len(data_2021)):
    if data_2021.iloc[i,14].month in summer_month: # 12 is  the data_time
        data_2021.iloc[i, -2] = 1
        
    if data_2021.iloc[i,14].month in holiday_month:
        data_2021.iloc[i, -1] = 1

In [ ]:
# rename columns to put into model

# for 2022, 2021 is the last 

data_test_2022 = data_2021.copy()

In [ ]:
data_test_2022.head()

In [ ]:
data_test_2022 = data_test_2022.loc[(data_test_2022.Month == 12)]

In [ ]:
data_test_2022.info()

In [ ]:
# data_test_2022 is only for 2022.01

data_test_2022.rename(columns={'SNAP_Applications':'last_snap','numberOfWorkers':'last_worker',\
                              'numberOfDisaster':'last_disaster', 'google_calfresh':'last_google_calfresh', \
                              'google_food_bank':'last_google_food_bank', 'google_food_pantry':'last_google_food_pantry',\
                              'google_food_stamps':'last_google_food_stamps', 'google_supplemental':'last_google_supplemental'}, inplace=True)

In [ ]:
test_Jan_2022 = data_test_2022.drop(columns={'State','google_snap','Year', 'Month', 'date_time', 'State_Texas' })

In [ ]:
test_Jan_2022.to_excel('test_Jan_2022.xlsx')

In [ ]:
new3 = new2.copy()

In [ ]:
col = list(new3.columns)

In [ ]:
col_need = ['State',
 'County',
 'fipsValue',
 'PDensity',
 'Population',
 'Year',
 'Month',
 'SNAP_Applications',
 'last_date_time',
 'date_time',
 'last_snap',
 'last_worker',
 'last_disaster',
 'last_google_calfresh',
 'last_google_food_bank',
 'last_google_food_pantry',
 'last_google_food_stamps',
 'last_google_supplemental',
 'last_google_snap']

In [ ]:
new3 = new3[col_need]
new3.head()

In [ ]:
new3.info()

In [ ]:
# this is for model training

new3.to_excel('DatabaseForFunction_2019_2021.xlsx')

### Adjust for Math model

#### Add one more feature - difference ratio for snap

### Adjust for Dashboard Json

-  add one variable ['snap_per_capita']
-  add one variable ['flag']

#### Add snap_per_capita

In [ ]:
df = new3.copy()

In [ ]:
# drop rows whose last_snap is 0

# Drop rows with 0 in column'snap_change_ratio'
df = df.drop(df.loc[(df['last_snap'] == 0)].index)

df['snap_per_capita'] = round((df.SNAP_Applications / df.last_snap),3)

df.head()

#### Add Flag

#### 1. For each county, calculate the mean and standard deviation for "snap per capita" across all data we have.

In [ ]:
gp = df.groupby(['State','County'])['snap_per_capita']

df1 = gp.agg({'mean','std'}).reset_index().rename(columns={'mean':'mean_snap_per_capita',
                                                          'std':'std_snap_per_capita'})

df1.head()

In [ ]:
df1['Yellow_flag'] = abs(df1.mean_snap_per_capita + df1.std_snap_per_capita)
df1['Red_flag'] = abs(df1.mean_snap_per_capita + 2*df1.std_snap_per_capita)

#### 2. Use mean + std dev for yellow and mean + 2*std dev for red but do this for the absolute values calculated by county above.

In [ ]:
# create a dict to lookup yellow and red flag for each county
# (State, County) : [yellow_value, red_value]
import collections

lookup = collections.defaultdict(list)

for i in range(len(df1)):
    state_key = df1.iloc[i,0]
    county_key = df1.iloc[i, 1]
    yellow = df1.iloc[i, -2]
    red = df1.iloc[i, -1]
    lookup[(state_key, county_key)] = [yellow, red]

In [ ]:
df.head()

In [ ]:
# flag county in df by the value in the lookup

# add one empty column for df
df['Flag'] = ''

# fill 'Flag' in df by the value in the lookup
for i in range(len(df)):
    state_key = df.iloc[i,0]
    county_key = df.iloc[i, 1]
    yellow = lookup[(state_key, county_key)][0]
    red = lookup[(state_key, county_key)][1]
    
    # check the value of snap_per_capita; idx = -2 cuz add one new column
    if df.iloc[i, -2] > red:
        df.iloc[i, -1] = 'Red'
    elif (df.iloc[i, -2] <= red) and (df.iloc[i, -2] > yellow):
        df.iloc[i, -1] = 'Yellow'
    else:
        df.iloc[i, -1] = 'Green'

In [ ]:
df.head()

### Create a json file - May 29


In [ ]:
col = df.columns.to_list()

In [ ]:
col.remove('snap_per_capita')

In [ ]:
df_json = df[col]

In [ ]:
df_json.info()

In [ ]:
df_json = df_json.to_dict(orient='records')

In [ ]:
import json

json_file = 'databaseForFunction_dashboard_2019_2021.json'

with open(json_file, 'w') as file:
    json.dump(df_json, file)

In [ ]:
# review json file

with open(json_file, 'r') as file:
    data = json.load(file)

# Dump the data to a formatted JSON string
formatted_json = json.dumps(data, indent=4, sort_keys=True)


In [ ]:
print(formatted_json)